# main

### 各セクションタイトル部分は先頭文字が数字になっているか確認＋woed内の各セルの末尾に余計な改行がないか確認

### automatic_translation.pyファイルの最後にあるif__name__=="__main__"以降のファイル名等を変更後に以下のセルを実行してください。

In [2]:
%run -i -t v5_automatic_translation.py

Total sensentces in this articl = 8
len(split_list): 2
i=0
split_list: ['Dendritic, i.e., tree-like, river networks are ubiquitous features on Earth’s landscapes; however, how and why river networks organize themselves into this form are incompletely understood. A branching pattern has been argued to be an optimal state. Therefore, we should expect models of river evolution to drastically reorganize (suboptimal) purely nondendritic networks into (more optimal) dendritic networks. To date, current physically based models of river basin evolution are incapable of achieving this result without substantial allogenic forcing. Here, we present a model that does indeed accomplish massive drainage reorganization. The key feature in our model is basin-wide lateral incision of bedrock channels. The addition of this submodel allows for channels to laterally migrate, which generates river capture events and drainage migration. An important factor in the model that dictates the rate and frequency o

In [ ]:
def _translation_deepl(self, sourse_texts=[]):

        """
        抜き出した英文をdeeplにコピペして翻訳する関数。引数は翻訳する文章が要素のリスト

        """

        res = requests.get('https://chromedriver.storage.googleapis.com/LATEST_RELEASE')
        browser = webdriver.Chrome(ChromeDriverManager(res.text).install())
        url = 'https://www.deepl.com/ja/translator'
        browser.get(url)
        # deepleにアクセスするまでしばらく待つ
        time.sleep(1)
        # wordの文章をparagraph単位で翻訳していく
        # 先頭が数字の場合にセクションのタイトルと判定している。
        translated_texts = []
        for i in range(len(sourse_texts)):         
            sourse_text = sourse_texts[i]

            print("i={}\n: {}\n".format(i, sourse_text))

            stextarea = browser.find_element_by_css_selector(
                '.lmt__textarea.lmt__source_textarea.lmt__textarea_base_style')
            ttextarea = browser.find_element_by_css_selector(
                '.lmt__textarea.lmt__target_textarea.lmt__textarea_base_style')
            
            lock.acquire()
            # stextarea.send_keys(sourse_text) #この方法だと絵文字（数式を送る事ができない）
            # javascriptを仕込む方法に変更 2022/4/22 https://tech.bita.jp/article/19
            INPUT_EMOJI = """
            arguments[0].value += arguments[1];
            arguments[0].dispatchEvent(new Event('change'));
            """
            browser.execute_script(INPUT_EMOJI, stextarea, sourse_text)
            lock.release()

            translated_text = ''

            lock.acquire()
            # 完全に翻訳されるまで繰り返す。判定は翻訳後文章の文字数が0かどうか
            # 文字数が0の場合は繰り返される。
            sec = self.wating_sec
            while not translated_text:
                # print("keep now....")
                time.sleep(sec)
                translated_text = ttextarea.get_property('value')
                print("translated_text: \n{}\n len: {}\n ".format(translated_text, len(translated_text))) 
                can_translated = re.search("\[\.\.\.\]", translated_text)
                if type(can_translated) is re.Match:
                    print("\ntry again\n")
                    translated_text = ''
                    sec += 2
                elif len(translated_text) == 0: # 翻訳されていない場合（[...]すら表示されていないパターン）
                    print("\ntry again\n")
                    translated_text = ''
                    sec += 2
                else:
                    if self.replace_flag_jp_to_jp:
                        translated_texts.append(self._keyword_replace(translated_text))
                    else:
                        translated_texts.append(translated_text)

            # ここに翻訳結果が出力されたら成功（途中経過をjupyter notebookで確認する場合はコメントイン）
            #print(translated_text)    
            lock.release()

            # sourse_textareaをクリアする
            time.sleep(2)
            stextarea.send_keys(Keys.CONTROL, "a")
            stextarea.send_keys(Keys.BACKSPACE)
            
        return translated_texts

In [3]:
from bs4 import BeautifulSoup
import requests

sourse_text = "こんにちは"

url = "https://www.deepl.com/ja/translator"

data = {
    "text": sourse_text,
    "source_lang": "EN",
    "target_lang": "JA"
}

res = requests.post(url, data=data)
soup = BeautifulSoup(res.text, 'html.parser')

translated = soup.find("div", {"class":"lmt__translations_as_text"})

if translated:
    translated_text = translated.text
    print(translated_text)
else:
    print("Element not found")



Element not found


In [ ]:
#headlessui-tabs-panel-7 > div > div.border-dark-7.border.bg-white.shadow-sm.xl\:rounded-lg > section > div > div.rounded-bl-inherit.rounded-br-inherit.grid.grid-cols-1.grid-rows-\[auto_auto\].min-\[768px\]\:grid-cols-\[1fr_auto_1fr\].min-\[768px\]\:grid-rows-\[1fr\].TextTranslatorLayout-module--bothareas--wXSVy > div.rounded-bl-inherit.relative.z-\[1\].min-h-\[240px\].min-w-0.md\:min-h-\[clamp\(250px\,50vh\,557px\)\] > section > div > div.relative.flex-1 > d-textarea > div:nth-child(1)

## end point